In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import os

In [2]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = datasets['train'], datasets['test']

In [3]:
strategy = tf.distribute.MirroredStrategy()
print('장치의 수:: {}'.format(strategy.num_replicas_in_sync))

장치의 수:: 1


In [4]:
#입력 파이프라인 구성
#데이터셋 내 샘플의 수는 info.splits.total_num_examples 로드
#얻을 수 있습니다..

num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA*strategy.num_replicas_in_sync

In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    
    return image, label

In [6]:
train_dataset = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [7]:
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])

In [8]:
#체크포인트를 저장할 체크포인트 디렉터리를 지정합니다..
checkpoint_dir ='./training_checkpoints'
#체크포인트 파일의 이름
checkpoint_prefix = os.path.join(checkpoint_dir,"ckpt_{epoch}")

In [9]:
#학습률을 점점 줄이기 위한 함수
#필요한 함수를 직접 정의하여 사용할 수 있습니다..
def decay(epoch):
    if epoch < 3:
        return 1e-3
    elif epoch >= 3 and epoch < 7:
        return 1e-4
    else:
        return 1e-5

In [10]:
class PrintLR(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print('\n에포크 {}의 학습률은 {}입니다'.format(epoch+1, model.optimizer.lr.numpy()))

In [13]:
# tf.keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1,profile_batch=3)
logdir = os.path.join('./mnist_logs')
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1,profile_batch=10000000),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

In [14]:
model.fit(train_dataset, epochs=12, callbacks=callbacks)

Epoch 1/12
    938/Unknown - 11s 12ms/step - loss: 0.1915 - accuracy: 0.9453
에포크 1의 학습률은 0.0010000000474974513입니다
938/938 [==============================] - 11s 12ms/step - loss: 0.1915 - accuracy: 0.9453
Epoch 2/12
931/938 [============================>.] - ETA: 0s - loss: 0.0678 - accuracy: 0.9791
에포크 2의 학습률은 0.0010000000474974513입니다
938/938 [==============================] - 9s 9ms/step - loss: 0.0677 - accuracy: 0.9791
Epoch 3/12
936/938 [============================>.] - ETA: 0s - loss: 0.0485 - accuracy: 0.9854
에포크 3의 학습률은 0.0010000000474974513입니다
938/938 [==============================] - 9s 10ms/step - loss: 0.0484 - accuracy: 0.9854
Epoch 4/12
937/938 [============================>.] - ETA: 0s - loss: 0.0271 - accuracy: 0.9926
에포크 4의 학습률은 9.999999747378752e-05입니다
938/938 [==============================] - 9s 9ms/step - loss: 0.0271 - accuracy: 0.9926
Epoch 5/12
934/938 [============================>.] - ETA: 0s - loss: 0.0236 - accuracy: 0.9936
에포크 5의 학습률은 9.999999747378752e-0

In [15]:
!ls {checkpoint_dir}

checkpoint
ckpt_1.data-00000-of-00002
ckpt_1.data-00001-of-00002
ckpt_1.index
ckpt_10.data-00000-of-00002
ckpt_10.data-00001-of-00002
ckpt_10.index
ckpt_11.data-00000-of-00002
ckpt_11.data-00001-of-00002
ckpt_11.index
ckpt_12.data-00000-of-00002
ckpt_12.data-00001-of-00002
ckpt_12.index
ckpt_2.data-00000-of-00002
ckpt_2.data-00001-of-00002
ckpt_2.index
ckpt_3.data-00000-of-00002
ckpt_3.data-00001-of-00002
ckpt_3.index
ckpt_4.data-00000-of-00002
ckpt_4.data-00001-of-00002
ckpt_4.index
ckpt_5.data-00000-of-00002
ckpt_5.data-00001-of-00002
ckpt_5.index
ckpt_6.data-00000-of-00002
ckpt_6.data-00001-of-00002
ckpt_6.index
ckpt_7.data-00000-of-00002
ckpt_7.data-00001-of-00002
ckpt_7.index
ckpt_8.data-00000-of-00002
ckpt_8.data-00001-of-00002
ckpt_8.index
ckpt_9.data-00000-of-00002
ckpt_9.data-00001-of-00002
ckpt_9.index


In [16]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
eval_loss, eval_acc = model.evaluate(eval_dataset)
print('평가 손실: {}, 평가 정확도: {}'.format(eval_loss, eval_acc))

157/157 [==============================] - 2s 15ms/step - loss: 0.0422 - accuracy: 0.9856417 - accu
평가 손실: 0.04224347562905828, 평가 정확도: 0.9855999946594238


In [19]:
!ls -sh ./logs

total 0
0 train


In [20]:
datasets

{'test': <_OptionsDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>,
 'train': <_OptionsDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>}